In [2]:
import redis, json
import pandas as pd
import utils

In [5]:
with open("../../.KEYS/DATA_SRC_INFO.json", "r") as f:
    storage_info = json.load(f)

In [7]:
redis_ip = storage_info['redis_conn_info']['ip']
redis_port = storage_info['redis_conn_info']['port']

redis_sassion = redis.StrictRedis(host=redis_ip, port=redis_port, db=0)

In [12]:
id_list = pd.read_csv("selected.csv").pid.to_list()

In [ ]:
def check_id_in_redis(logger, redis_session, records):
    inited_id_records = []
    for record in records:
        _id = record.get('id')
        _get_date = record.get('get_date')
        if not redis_session.hexists('compare_id', _id):  # Redis hash에 해당 id가 없으면
            inited_id_records.append({"id": _id, "get_date": _get_date})
            logger.info(f"id {_id} is not exist in redis. Added upload list.")
        else:
            logger.info(f"id {_id} already exist in redis. This id passed.")
    
    logger.info(f"Completed check ids: target ids are {inited_id_records}")
    return inited_id_records

# record에서 없는 id만 redis로 push
def upload_id_into_redis(logger, redis_session, records):
    for record in records:
        _id = record.get('id')
        _get_date = record.get('get_date')
        if not redis_session.hexists('compare_id', _id):  # Redis hash에 해당 id가 없으면
            redis_session.hset('compare_id', _id, _get_date)  # 해당 id를 해시에 저장
            logger.info(f"id {_id} init into redis.")
        else:
            logger.info(f"id {_id} already exist in redis. set id action dismissed")